In [1]:
import os
import faiss
import torch
print(torch.cuda.is_available())

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore

True


In [ ]:
from dotenv import load_dotenv
load_dotenv("../keys.env")

openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

In [2]:
embed_model_name = "intfloat/multilingual-e5-large-instruct"
index_path = "./indexes/2025-04-08-06-54-34"


model_kwargs = {
    "device": "cuda:1",
    "trust_remote_code": True
}
encode_kwargs = {
    "normalize_embeddings": True,
    "batch_size": 128,
    "multi_process": True,
    "show_progress": True
}

embedding_model = HuggingFaceEmbeddings(
    model_name=embed_model_name, 
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
vector_store = FAISS.load_local(index_path, embeddings=embedding_model, allow_dangerous_deserialization=True)

In [15]:
"""
reference : https://wikidocs.net/234016 

vectore_store에 대한 검색기(retriever)를 생성.
 - search_type : 검색 유형. similarity, mmr(Maximal Marginal Relevance), similarity_score_threshold 중 1개
 - search_kwargs : 추가 검색 옵션
    - k : 반환할 문서 수(기본=4)
    - score_threshold : 유사도 임계치(기본=0.0)
    - fetch_k : MMR 알고리즘에 전달할 문서 수(기본=20)
    - lambda_mult : MMR 결과의 다양성을 조절.(0~1사이 값으로 기본값=0.5)
    - filter : 문서 메타데이터 기반 필터링 --> 메타데이터 필터링으로 특정 조건의 문서만 검색 가능.
"""

retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 10, "score_threshold": 0.3}
)

ret_results = retriever.invoke("코리아교육그룹의 비전, 미래, 목표")

for ret in ret_results:
    print(ret.metadata['company_name'], ret.metadata['url'], ret.metadata['original_doc_id'])
    print("="*100)
    print(ret.page_content)

코리아교육그룹 http://www.koreaedugroup.com/2018/about/ceo.asp e9f7d6c0-e4b5-4c91-a221-6654e3c842e2
EDUCATION
COMPANY
CAREERS
CEO MESSAGE
HISTORY
PERFORMANCE
EDUCATION SHARING
GROUP ACTIVITIES
오늘보다 더 나은 내일
A bright future for Korean education
FOR BETTER FUTURE
미래를 만드는 꿈의 공방
코리아교육그룹은 ‘미래를 만드는 꿈의 공방’을 모토로 교육이 곧 미래의 비전이라는 설립취지에 따라 교육서비스를 핵심사업으로 성장해 가고 있는 전문 기업입니다. 저희 코리아교육그룹은 10여 년 전 컴퓨터그래픽과 디자인 교육을 시작으로 온라인 원격교육과 뷰티관련 교육, 항공승무원 및 서비스인력 양성교육 등 다양한 교육서비스의 확장을 통해 전문 인력을 양성해 왔습니다.

코리아교육그룹 전 임직원들은 개인의 미래를 개척하고 더 나아가 국가의 미래를 이끌어 갈 인재를 양성한다는 사명감과 자부심으로 항상 새로운 이상과 목표를 향해 나아가고 있습니다. 무한 경쟁의 시대, 코리아교육그룹은 남과의 경쟁이 아닌 지금껏 걸어온 자신과의 경쟁이 우선임을 자각하고 높은 이상의 실현을 위해 일신우일신(日新又日新)하는 교육기업이 되겠습니다.

교육서비스의 더 나은 미래를 위해 전 임직원이 함께 움직이는 코리아교육그룹의 힘찬 발걸음을 성원하고 지켜봐 주시기 바랍니다. 감사합니다.
코리아교육그룹 대표이사
김영우
그룹소개
찾아오시는길
인재채용
서울 강남구 강남대로 286 부영빌딩 3, 4층 (우)06258 대표이사 김영우
© KOREA EDUCATION GROUP. All Rights Reserved.
코리아교육그룹 http://www.koreaedugroup.com/2018/careers/talents.asp 21ab53d8-7415-4552-96c6-bf7801c1a164
EDUCATION
COMPANY
CAREERS

In [16]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "주어진 기업이 설정한 비전, 목표, 미래에 대한 내용들을 추출하고 정리해서 문장형태로 서술해주세요."
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", "{input_documents}")
])

str_ret_results = [ret.page_content for ret in ret_results]
prompt = prompt_template.invoke({"input_documents": str_ret_results})
print(prompt)

messages=[SystemMessage(content='주어진 기업이 설정한 비전, 목표, 미래에 대한 내용들을 추출하고 정리해서 문장형태로 서술해주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='[\'EDUCATION\\nCOMPANY\\nCAREERS\\nCEO MESSAGE\\nHISTORY\\nPERFORMANCE\\nEDUCATION SHARING\\nGROUP ACTIVITIES\\n오늘보다 더 나은 내일\\nA bright future for Korean education\\nFOR BETTER FUTURE\\n미래를 만드는 꿈의 공방\\n코리아교육그룹은 ‘미래를 만드는 꿈의 공방’을 모토로 교육이 곧 미래의 비전이라는 설립취지에 따라 교육서비스를 핵심사업으로 성장해 가고 있는 전문 기업입니다. 저희 코리아교육그룹은 10여 년 전 컴퓨터그래픽과 디자인 교육을 시작으로 온라인 원격교육과 뷰티관련 교육, 항공승무원 및 서비스인력 양성교육 등 다양한 교육서비스의 확장을 통해 전문 인력을 양성해 왔습니다.\\n\\n코리아교육그룹 전 임직원들은 개인의 미래를 개척하고 더 나아가 국가의 미래를 이끌어 갈 인재를 양성한다는 사명감과 자부심으로 항상 새로운 이상과 목표를 향해 나아가고 있습니다. 무한 경쟁의 시대, 코리아교육그룹은 남과의 경쟁이 아닌 지금껏 걸어온 자신과의 경쟁이 우선임을 자각하고 높은 이상의 실현을 위해 일신우일신(日新又日新)하는 교육기업이 되겠습니다.\\n\\n교육서비스의 더 나은 미래를 위해 전 임직원이 함께 움직이는 코리아교육그룹의 힘찬 발걸음을 성원하고 지켜봐 주시기 바랍니다. 감사합니다.\\n코리아교육그룹 대표이사\\n김영우\\n그룹소개\\n찾아오시는길\\n인재채용\\n서울 강남구 강남대로 286 부영빌딩 3, 4층 (우)06258 대표이사 김영우\\n© KOREA EDUCATION GROUP. All Rights Reserved.\', \'EDUCA

In [20]:
from langchain_anthropic import ChatAnthropic

chat_model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0, api_key=anthropic_api_key)
response = chat_model.invoke(prompt)
print(response.content)

코리아교육그룹의 비전, 목표, 미래에 대한 주요 내용을 정리하면 다음과 같습니다:

1. 비전: "미래를 만드는 꿈의 공방"을 모토로 하여 교육을 통해 미래의 비전을 실현하고자 합니다.

2. 목표: 
- 교육서비스를 핵심사업으로 성장하는 전문 교육기업이 되는 것
- 개인의 미래를 개척하고 국가의 미래를 이끌어갈 인재 양성
- 대한민국에서 100년을 넘어가는 교육기업으로 성장

3. 미래 방향:
- 다양한 교육 분야(컴퓨터, 디자인, IT, 게임, 뷰티, 요리 등)로 교육서비스 확장 
- 혁신적이고 창의적인 교육 프로그램 개발
- 실무 중심의 교육을 통한 취업 연계 강화
- 교육나눔 활동을 통한 사회공헌 확대
- 글로벌 경쟁력을 갖춘 교육기업으로 도약

4. 핵심 가치:
- 창의성, 전문성, 혁신, 도전정신 추구
- 미래지향적 사고와 지속적인 성장 지향
- 교육을 통한 사회 발전 기여

코리아교육그룹은 교육을 통해 개인과 사회의 미래를 선도하는 혁신적인 교육기업으로 성장하고자 하는 비전과 목표를 가지고 있습니다.


In [21]:
from langchain.prompts import ChatPromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnableLambda

company_name = "LG CNS"

# 1. 시스템 프롬프트 템플릿 정의
system_template = "주어진 기업이 설정한 비전, 목표, 미래에 대한 내용들을 추출하고 정리해서 문장형태로 서술해주세요."
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", "{input_documents}")
])

# 2. 벡터 저장소 → retriever (Runnable)
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 10, "score_threshold": 0.3}
)

# 3. 검색 결과에서 page_content만 추출하는 함수 (Runnable로 래핑)
extract_content = RunnableLambda(lambda docs: {"input_documents": [doc.page_content for doc in docs]})

# 4. LLM 모델
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0, api_key=anthropic_api_key)

# 5. 체인 구성: retriever → page_content 추출 → prompt → llm
chain = retriever | extract_content | prompt_template | llm

# 6. 실행
response = chain.invoke(f"{company_name}의 비전, 미래, 목표")
print(f"{company_name}에 대한 응답:\n{response.content}")


LG CNS에 대한 응답:
LG CNS의 비전, 목표, 미래에 대한 내용을 정리하면 다음과 같습니다:

LG CNS는 지속가능경영을 강화하고 기업시민으로서 사회적 책임을 다하며 지속적으로 성장 가능한 DX(디지털 전환) 전문기업이 되고자 합니다. 회사는 고객과 함께 성장하는 최고의 DX 파트너가 되겠다는 목표를 가지고 있습니다. 

LG CNS는 국내 대표 DX기업을 넘어 글로벌 시장까지 아우르는 'AX Company'로 도약하고자 합니다. 고객가치를 혁신할 수 있는 새로운 기술과 서비스를 발굴/확보하여, 고객이 필요로 하는 AI기술을 가장 빠르고 적절하게 제공하는 'AX 전문 기업'으로서 시장을 선도하겠다는 비전을 제시하고 있습니다.

또한 LG CNS는 고객에게 가장 신뢰받는 'Digital Business Innovator'로 자리매김하고자 합니다. 독보적인 DX 전문성과 차별화된 서비스로 고객의 비즈니스 효율성을 제고하며 국내 DX 시장을 선도하고 있습니다. 

회사는 클라우드, AI, 빅데이터 등 IT 신기술을 바탕으로 4차 산업혁명 시대의 DX를 선도하고 있으며, 앞으로도 이러한 노력을 지속할 것임을 밝히고 있습니다.
